# NativQA: Text, Image, and Video Search

This notebook runs the NativQA CLI for all supported search types: `text`, `image`, and `video`.

Before running:
- Put your SerpAPI key in `envs/api_key.env` as `API_KEY="..."`.
- Install requirements in your environment.

In [ ]:
from pathlib import Path
import subprocess
import shlex

# Assumes notebook is in <repo>/examples/
REPO_ROOT = Path.cwd().resolve().parent
DATA_FILE = REPO_ROOT / "data" / "test_query.csv"
ENV_FILE = REPO_ROOT / "envs" / "api_key.env"
OUTPUT_BASE = REPO_ROOT / "results"

COUNTRY_CODE = "us"
LOCATION = "New York, United States"
N_ITER = 1

print(f"REPO_ROOT: {REPO_ROOT}")
print(f"DATA_FILE exists: {DATA_FILE.exists()} -> {DATA_FILE}")
print(f"ENV_FILE exists: {ENV_FILE.exists()} -> {ENV_FILE}")
print(f"OUTPUT_BASE: {OUTPUT_BASE}")

In [ ]:
def run_nativqa(search_type: str, engine: str = "google"):
    cmd = [
        "python3", "-m", "nativqa",
        "--engine", engine,
        "--search_type", search_type,
        "--input_file", str(DATA_FILE),
        "--country_code", COUNTRY_CODE,
        "--location", LOCATION,
        "--env", str(ENV_FILE),
        "--output_dir", str(OUTPUT_BASE),
        "--n_iter", str(N_ITER),
    ]

    print("Running command:")
    print(" \n".join([shlex.join(cmd)]))
    result = subprocess.run(
        cmd,
        cwd=REPO_ROOT,
        text=True,
        capture_output=True,
    )

    print("\nExit code:", result.returncode)
    if result.stdout:
        print("\nSTDOUT (tail):")
        print("\n".join(result.stdout.splitlines()[-40:]))
    if result.stderr:
        print("\nSTDERR (tail):")
        print("\n".join(result.stderr.splitlines()[-40:]))

    if result.returncode != 0:
        raise RuntimeError(f"nativqa failed for search_type={search_type}")

    out_dir = OUTPUT_BASE / search_type / DATA_FILE.stem
    print(f"\nExpected output directory: {out_dir}")
    return out_dir


## 1) Text Search

In [ ]:
text_out = run_nativqa("text")
text_out

## 2) Image Search

In [ ]:
image_out = run_nativqa("image")
image_out

## 3) Video Search

In [ ]:
video_out = run_nativqa("video")
video_out

## Quick Output Check

In [ ]:
for p in [text_out, image_out, video_out]:
    print("\n", p)
    if p.exists():
        for child in sorted(p.rglob("*")):
            if child.is_file():
                print(" -", child.relative_to(p))
    else:
        print(" - not created")